In [1]:
import gym
from gym import spaces
import pybullet as p
import pybullet_data
import numpy as np


In [2]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import SAC

In [3]:
import gymnasium as gym
from gymnasium import spaces
import pybullet as p
import pybullet_data
import numpy as np
import time

class RoboticDogEnv(gym.Env):
    def __init__(self):
        super(RoboticDogEnv, self).__init__()
        self.total_reward=0
        self.counter=0
        self.steps=0
        self.total_steps=0
        # Connect to PyBullet
        self.physicsClient = p.connect(p.GUI)

        # Set the simulation environment
        p.setAdditionalSearchPath(pybullet_data.getDataPath())

        # Load the plane and URDF
        self.plane_id = p.loadURDF("plane.urdf")

        # Define the URDF path as an instance attribute
        self.urdf_path = "C:/Users/youss/OneDrive - aucegypt.edu/Desktop/bolt/spot.urdf"
        self.robot_id = p.loadURDF(self.urdf_path, [0, 0, 0.15], useFixedBase=False)
        self.num_joints = p.getNumJoints(self.robot_id)

        # Ensure the robot is correctly initialized
        if self.robot_id < 0:
            raise ValueError("Failed to initialize the robot.")
        
        if self.num_joints == 0:
            raise ValueError("No joints found in the robot.")
        
        print(f"Robot ID: {self.robot_id}")
        print(f"Number of joints: {self.num_joints}")
        
        # Set gravity
        p.setGravity(0, 0, -9.8)
        
        # Get joint limits and ensure joints are valid
        self.joint_indices = []
        
        joint_limits = []
        for joint_index in range(self.num_joints):
            joint_info = p.getJointInfo(self.robot_id, joint_index)
            joint_name = joint_info[1].decode('utf-8')
            joint_lower_limit = joint_info[8]
            joint_upper_limit = joint_info[9]
            
            # Print joint information for debugging
           # print(f"Joint {joint_index}: Name: {joint_name}, Lower limit: {joint_lower_limit}, Upper limit: {joint_upper_limit}")
            
            if "coxa" in joint_name or "femur" in joint_name or "tibia" in joint_name:
                # Add fallback default limits if the provided limits are invalid
                if joint_lower_limit >= joint_upper_limit:
                    joint_lower_limit, joint_upper_limit = -1.0, 1.0
                
                joint_limits.append((joint_lower_limit, joint_upper_limit))
                self.joint_indices.append(joint_index)

                
        
                
        # Define the action and observation space
        self.action_space = spaces.Box(
            low=np.array([limit[0] for limit in joint_limits]),
            high=np.array([limit[1] for limit in joint_limits]),
            dtype=np.float32
        )
        
        self.observation_space = spaces.Box(
            low=-np.inf,
            high=np.inf,
            shape=(len(self.joint_indices) * 2 + 6,),  # Joint positions + velocities + base orientation (3) + base angular velocity (3)
            dtype=np.float32
        )
        
        self.fall_start_time = None  # Initialize the fall start time
        self.leg_indices = self.joint_indices   
        self.noise_counter = 0 
        
        self.forward=0
        self.stable=0
        self.energy=0
        self.velocity_pen=0
        self.fall=0
        self.smooth=0
        self.previous_joint_velocities = None
        self.sym=0
        
       
        
        # Store the link indices for the feet
        self.front_left_foot = self.get_link_index('foot_FL')
        self.front_right_foot = self.get_link_index('foot_FR')
        self.back_left_foot = self.get_link_index('foot_BL')
        self.back_right_foot = self.get_link_index('foot_BR')
        
        
        
        
    def get_link_index(self, link_name):
        for joint_index in range(self.num_joints):
            joint_info = p.getJointInfo(self.robot_id, joint_index)
            joint_name = joint_info[12].decode('utf-8')  # Link name is at index 12
            if link_name in joint_name:
                return joint_info[0]  # Return the link index
        raise ValueError(f"Link {link_name} not found")
        
        
    def reset(self, **kwargs):
        self.counter=self.counter+1
        self.total_steps=self.steps+self.total_steps
        if self.steps!=0: 
            average = self.total_reward/self.steps
        if self.total_reward != 0:
            print(f"Episode {self.counter}  finished with cumulative reward: {self.total_reward} and ")
            print(f"with an average reward of: {average}")
            print(f"number of steps in this episode: {self.steps}")
            print(f"total steps till now: {self.total_steps}")
            print(f"..........................................")
        self.total_reward=0
        self.steps=0
        # Reset the simulation environment
        p.resetSimulation()
        p.setAdditionalSearchPath(pybullet_data.getDataPath())
        self.plane_id = p.loadURDF("plane.urdf")
        self.robot_id = p.loadURDF(self.urdf_path, [0, 0, 0.15], useFixedBase=False)
        p.setGravity(0, 0, -9.8)
        
        # Wait for the simulation to stabilize
        time.sleep(1)
        
        # Ensure the robot is correctly initialized
        if self.robot_id < 0:
            print("Error: Robot failed to load.")
            return None, {}
        
        # Check if joints are correctly initialized
        if p.getNumJoints(self.robot_id) == 0:
            print("Error: No joints found in the robot.")
            return None, {}
        
        # Return the initial state
        initial_state = self.get_state()
       # print("Initial state:", initial_state)  # Debugging statement
        self.fall_start_time = None  # Reset the fall start time
        
        
        print(f"forward : {self.forward/2500} \n velocity penalty:{self.velocity_pen/2500} \n stability : {self.stable/2500}")
        print(f"energy : {self.energy/2500} \n  fall: {self.fall/2500} \n smooth: {self.smooth/2500} \n symmetry: {self.sym/2500}")      
                 
                 
                 
                 
                 
        self.forward=0
        self.stable=0
        self.energy=0 
        self.velocity_pen=0
        self.fall=0
        self.smooth=0
        self.sym=0
       
        
        return initial_state, {}
    
    def step(self, action):
        # Apply action to the robot
        for i, joint_index in enumerate(self.joint_indices):
            p.setJointMotorControl2(self.robot_id, joint_index, p.POSITION_CONTROL, targetPosition=action[i])
        
        # Step the simulation
        p.stepSimulation()
        time.sleep(1. / 240.)  # Adjust sleep time to control simulation speed
        
        # Get the next state, reward, and check if done
        state = self.get_state()
        reward = self.compute_reward()
        terminated = self.is_done(state)
        truncated = False  # This can be set to True based on some logic, e.g., max steps reached
        self.total_reward=self.total_reward+reward
        self.steps=self.steps+1
        return state, reward, terminated, truncated, {}
    
    def get_state(self):
        # Debugging: Check joint indices
       # print("Joint indices:", self.joint_indices)
        
        joint_states = p.getJointStates(self.robot_id, self.joint_indices)
        if joint_states is None or len(joint_states) == 0:
           # print("Error: p.getJointStates returned None or an empty list")  # Debugging statement
            return None
        
        joint_positions = [state[0] for state in joint_states]
        joint_velocities = [state[1] for state in joint_states]
        
        base_position, base_orientation = p.getBasePositionAndOrientation(self.robot_id)
        base_velocity, base_angular_velocity = p.getBaseVelocity(self.robot_id)
        
        base_orientation_euler = p.getEulerFromQuaternion(base_orientation)
        
        state = np.concatenate([joint_positions, joint_velocities, base_orientation_euler, base_angular_velocity])
        
        # Check for nan values in the state
        if np.isnan(state).any():
            print("Error: State contains NaN values")
            print("Joint positions:", joint_positions)
            print("Joint velocities:", joint_velocities)
            print("Base orientation (Euler):", base_orientation_euler)
            print("Base angular velocity:", base_angular_velocity)
        
        return state
    
    
    def compute_reward(self): #under testing 
        base_position, base_orientation = p.getBasePositionAndOrientation(self.robot_id)
        base_velocity, base_angular_velocity = p.getBaseVelocity(self.robot_id)
        
        
    
        # Calculate the forward vector from the orientation
        forward_vector = p.getMatrixFromQuaternion(base_orientation)[0:3:2]
        forward_vector = np.array(forward_vector)  # Forward direction in the world frame
        forward_velocity = np.dot(forward_vector, base_velocity[:2])
        forward_reward = forward_velocity
        
       
         # Penalize high velocities
        max_velocity_threshold = 3.0  # Set the maximum desired velocity
        if forward_velocity > max_velocity_threshold:
            velocity_penalty = (forward_velocity - max_velocity_threshold) ** 2
        else:
            velocity_penalty = -1
            
        
        
        
        base_orientation_euler = p.getEulerFromQuaternion(base_orientation)
        pitch, roll = base_orientation_euler[1], base_orientation_euler[0]
        # Set acceptable thresholds for instability
        acceptable_pitch_threshold = 0.05  # Adjust as necessary
        acceptable_roll_threshold = 0.05  # Adjust as necessary
 
        stability_penalty = 0.0
        stability_penalty += abs(pitch) - acceptable_pitch_threshold
        stability_penalty += abs(roll) - acceptable_roll_threshold
        # Penalize instability
        
       
        """"
        noise = 0.0
        self.noise_counter += 1
        if self.noise_counter % 100 == 0:  # Add noise every 100 steps
            noise = np.random.normal(0, 1)  # Adjust the standard deviation as needed
        """
        
        joint_states = p.getJointStates(self.robot_id, self.joint_indices)
        joint_angles = [state[0] for state in joint_states]
        joint_velocities = [state[1] for state in joint_states]
        # Apply dynamic energy penalty
        energy_penalty = 0.0
        joint_velocity_threshold = 10  # Threshold for penalizing high joint velocities
        for velocity in joint_velocities:
            if abs(velocity) > joint_velocity_threshold:
                energy_penalty += (abs(velocity) - joint_velocity_threshold) ** 2 
            else: energy_penalty= -100
        # Penalize high joint velocities
        
        
        
    
        fall_penalty = 0.0
       
        if base_position[2] < 0.12:
         # If the robot has fallen (height threshold)
            fall_penalty = 50.0 
        else: fall_penalty= -1
        
        
        smoothness_penalty = self.compute_smoothness_reward(joint_velocities)
        
        symmetry_penalty= self.compute_symmetry_reward()
        
        
        # Scaling factors
        forward_scale = 0#700
        stability_penalty_scale =  0#-15 #was 0.2
        energy_penalty_scale =  0#-0.1
        fall_penalty_scale =  -500.0
        
        smoothness_scale= 0# -0.05
        symmetry_penalty_scale = 0#5.0
        velocity_penalty_scale = 0
        
        
        reward = (forward_reward * forward_scale) + \
                 (velocity_penalty * velocity_penalty_scale)+ \
                 (energy_penalty*energy_penalty_scale) +\
                 (stability_penalty * stability_penalty_scale) + \
                 (fall_penalty * fall_penalty_scale) + \
                 (smoothness_penalty*smoothness_scale) +\
                 (symmetry_penalty * symmetry_penalty_scale)
               
                 
            
            
            
            
        self.forward=self.forward+(forward_reward*forward_scale)
        self.velocity_pen=self.velocity_pen+(velocity_penalty * velocity_penalty_scale)
        self.stable=self.stable+(stability_penalty*stability_penalty_scale)
        self.energy=(energy_penalty*energy_penalty_scale)+self.energy
        self.fall=self.fall+(fall_penalty * fall_penalty_scale)
        self.smooth=self.smooth+(smoothness_penalty*smoothness_scale)
        self.sym=self.sym+(symmetry_penalty * symmetry_penalty_scale)  
            
        
        return reward
    
    
    def compute_symmetry_reward(self):
        # Get the height of each foot joint
        front_left_height = p.getLinkState(self.robot_id, self.front_left_foot)[0][2]
        front_right_height = p.getLinkState(self.robot_id, self.front_right_foot)[0][2]
        back_left_height = p.getLinkState(self.robot_id, self.back_left_foot)[0][2]
        back_right_height = p.getLinkState(self.robot_id, self.back_right_foot)[0][2]
        
        # Reward for diagonal leg coordination
        symmetry_reward = 0.0
        if abs(front_left_height - back_right_height) < 0.015:  # Adjust threshold as needed
            symmetry_reward += 1.0
        if abs(front_right_height - back_left_height) < 0.015:  # Adjust threshold as needed
            symmetry_reward += 1.0

        return symmetry_reward

    
    def compute_smoothness_reward(self, joint_velocities):
        velocity_change_threshold = 2
        smoothness_penalty=0
        if self.previous_joint_velocities is None:
            smoothness_reward = 0.0
        else:
            velocity_differences = np.abs(np.array(joint_velocities) - np.array(self.previous_joint_velocities))
            velocity_differences = np.clip(velocity_differences - velocity_change_threshold, 0, None)
            smoothness_penalty = np.sum(velocity_differences)
            

        self.previous_joint_velocities = joint_velocities
        return smoothness_penalty

   


    
    
    def is_done(self, state):
        """"
        base_position, base_orientation = p.getBasePositionAndOrientation(self.robot_id)
        
        if base_position[2] < 0.2:  # If the robot has fallen
            if self.fall_start_time is None:
                self.fall_start_time = time.time()  # Start the timer
               # print("Robot has started falling")
            elif time.time() - self.fall_start_time > 2:  # Wait for 2 seconds before deciding to reset
               # print("Robot has fallen for 2 seconds, resetting...")
                return True
        else:
            self.fall_start_time = None  # Reset the timer if the robot is not fallen
        """
         
        if self.steps == 2500:
            return True 
        
        else:
            return False
    
    def render(self, mode='human'):
        pass
    
    def close(self):
        p.disconnect()

    


In [4]:
# Register the environment
gym.envs.registration.register(id='RoboticDog-v0', entry_point='__main__:RoboticDogEnv')


In [5]:

# Create the environment
env = RoboticDogEnv()


Robot ID: 1
Number of joints: 16


In [6]:

# Instantiate the agent
#model = PPO('MlpPolicy', env, verbose=1)
# for more exploration learning rate could co down till 1e-4 and entropy could go up till 0.1

model = PPO('MlpPolicy', env, verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.



# Train the agent
model.learn(total_timesteps=1000000000)


# Load the previously trained model
model = PPO.load("C:\\Users\\youss\\OneDrive - aucegypt.edu\\Desktop\\bolt\\ppo_spot_trial_basic.zip", env=env)


In [7]:
# Train the agent
model.learn(total_timesteps=5000000)

forward : 0.0 
 velocity penalty:0.0 
 stability : 0.0
energy : 0.0 
  fall: 0.0 
 smooth: 0.0 
 symmetry: 0.0
-----------------------------
| time/              |      |
|    fps             | 89   |
|    iterations      | 1    |
|    time_elapsed    | 22   |
|    total_timesteps | 2048 |
-----------------------------
Episode 2  finished with cumulative reward: -3441500.0 and 
with an average reward of: -1376.0495801679328
number of steps in this episode: 2501
total steps till now: 2501
..........................................
forward : 0.0 
 velocity penalty:0.0 
 stability : 0.0
energy : 0.0 
  fall: -1376.6 
 smooth: 0.0 
 symmetry: 0.0
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 2.5e+03       |
|    ep_rew_mean          | -3.44e+06     |
| time/                   |               |
|    fps                  | 80            |
|    iterations           | 2             |
|    time_elapsed         | 50           

Episode 8  finished with cumulative reward: -5966000.0 and 
with an average reward of: -2385.4458216713315
number of steps in this episode: 2501
total steps till now: 17507
..........................................
forward : 0.0 
 velocity penalty:0.0 
 stability : 0.0
energy : 0.0 
  fall: -2386.4 
 smooth: 0.0 
 symmetry: 0.0
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 2.5e+03      |
|    ep_rew_mean          | -7.3e+06     |
| time/                   |              |
|    fps                  | 73           |
|    iterations           | 9            |
|    time_elapsed         | 249          |
|    total_timesteps      | 18432        |
| train/                  |              |
|    approx_kl            | 9.654323e-07 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -17          |
|    explained_variance   | 1.03e-05     |
|    learning_rate      

Episode 14  finished with cumulative reward: -1605500.0 and 
with an average reward of: -641.9432227109156
number of steps in this episode: 2501
total steps till now: 32513
..........................................
forward : 0.0 
 velocity penalty:0.0 
 stability : 0.0
energy : 0.0 
  fall: -642.2 
 smooth: 0.0 
 symmetry: 0.0
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 2.5e+03       |
|    ep_rew_mean          | -6.79e+06     |
| time/                   |               |
|    fps                  | 71            |
|    iterations           | 16            |
|    time_elapsed         | 458           |
|    total_timesteps      | 32768         |
| train/                  |               |
|    approx_kl            | 7.7550067e-07 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -17           |
|    explained_variance   | 1.65e-05      |
|    learn

-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 2.5e+03       |
|    ep_rew_mean          | -6.57e+06     |
| time/                   |               |
|    fps                  | 71            |
|    iterations           | 23            |
|    time_elapsed         | 661           |
|    total_timesteps      | 47104         |
| train/                  |               |
|    approx_kl            | 1.5369442e-06 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -17           |
|    explained_variance   | 1.51e-05      |
|    learning_rate        | 0.0003        |
|    loss                 | 3.39e+09      |
|    n_updates            | 220           |
|    policy_gradient_loss | -0.000266     |
|    std                  | 1             |
|    value_loss           | 9.94e+09      |
-------------------------------------------
Episode 20  finished with cumula

Episode 25  finished with cumulative reward: -4793000.0 and 
with an average reward of: -1916.4334266293483
number of steps in this episode: 2501
total steps till now: 60024
..........................................
forward : 0.0 
 velocity penalty:0.0 
 stability : 0.0
energy : 0.0 
  fall: -1917.2 
 smooth: 0.0 
 symmetry: 0.0
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 2.5e+03      |
|    ep_rew_mean          | -6.33e+06    |
| time/                   |              |
|    fps                  | 70           |
|    iterations           | 30           |
|    time_elapsed         | 873          |
|    total_timesteps      | 61440        |
| train/                  |              |
|    approx_kl            | 6.923801e-08 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -17          |
|    explained_variance   | -1.93e-05    |
|    learning_rate     

...

forward : 0.0 
 velocity penalty:0.0 
 stability : 0.0
energy : 0.0 
  fall: -3324.8 
 smooth: 0.0 
 symmetry: 0.0
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 2.5e+03   |
|    ep_rew_mean          | -6.08e+06 |
| time/                   |           |
|    fps                  | 74        |
|    iterations           | 2014      |
|    time_elapsed         | 55093     |
|    total_timesteps      | 4124672   |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -17       |
|    explained_variance   | 0.0337    |
|    learning_rate        | 0.0003    |
|    loss                 | 1.1e+10   |
|    n_updates            | 20130     |
|    policy_gradient_loss | -1.64e-06 |
|    std                  | 1         |
|    value_loss           | 1.96e+10  |
---------------------------------------
Episo

Episode 1656  finished with cumulative reward: -5405000.0 and 
with an average reward of: -2161.135545781687
number of steps in this episode: 2501
total steps till now: 4139155
..........................................
forward : 0.0 
 velocity penalty:0.0 
 stability : 0.0
energy : 0.0 
  fall: -2162.0 
 smooth: 0.0 
 symmetry: 0.0
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 2.5e+03   |
|    ep_rew_mean          | -6.11e+06 |
| time/                   |           |
|    fps                  | 74        |
|    iterations           | 2022      |
|    time_elapsed         | 55304     |
|    total_timesteps      | 4141056   |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -17       |
|    explained_variance   | 0.0395    |
|    learning_rate        | 0.0003    |
|    loss                

Episode 1662  finished with cumulative reward: -9179000.0 and 
with an average reward of: -3670.1319472211117
number of steps in this episode: 2501
total steps till now: 4154161
..........................................
forward : 0.0 
 velocity penalty:0.0 
 stability : 0.0
energy : 0.0 
  fall: -3671.6 
 smooth: 0.0 
 symmetry: 0.0
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 2.5e+03   |
|    ep_rew_mean          | -6.09e+06 |
| time/                   |           |
|    fps                  | 74        |
|    iterations           | 2029      |
|    time_elapsed         | 55495     |
|    total_timesteps      | 4155392   |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -17       |
|    explained_variance   | 0.0358    |
|    learning_rate        | 0.0003    |
|    loss               

Episode 1668  finished with cumulative reward: -2600000.0 and 
with an average reward of: -1039.5841663334666
number of steps in this episode: 2501
total steps till now: 4169167
..........................................
forward : 0.0 
 velocity penalty:0.0 
 stability : 0.0
energy : 0.0 
  fall: -1040.0 
 smooth: 0.0 
 symmetry: 0.0
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 2.5e+03   |
|    ep_rew_mean          | -5.97e+06 |
| time/                   |           |
|    fps                  | 74        |
|    iterations           | 2036      |
|    time_elapsed         | 55684     |
|    total_timesteps      | 4169728   |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -17       |
|    explained_variance   | 0.049     |
|    learning_rate        | 0.0003    |
|    loss               

error: Not connected to physics server.

In [ ]:
# Save the model with a specific name
model.save("ppo_spot_trial_basic2")

print("Model saved successfully as trial_one.zip.")
